In [72]:
import requests
import urllib
import re
import threading
from bs4 import *
import socket
import time


authors = ['https://book.douban.com/author/1071735/','https://book.douban.com/author/4514938/']
books = ['5431784', '26986954', '4820710', '25967870', '26279019']
def get_books():
    
    global books

    #从你的浏览器控制台复制出http报文的header信息
    header = {'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
    for i in range(0,40,20):
        url = 'https://book.douban.com/tag/%E5%B0%8F%E8%AF%B4?start='+str(i)+'&type=T'
        #print(url)
        #发送一个http请求，读出网页内容存到html
        req = urllib.request.Request(url,headers=header)
        html = urllib.request.urlopen(req).read()
        time.sleep(3)
        #网页里有中文，需要decode
        html.decode('utf-8','ignore')

        soup = BeautifulSoup(html,'lxml')


        links = soup.find_all('a')
        for link in links:
            href = link.get('href')
            if re.match(r'https://book.douban.com/subject/[0-9]*/$',href):
                bookID = re.search(r'https://book.douban.com/subject/([0-9]*)/$',href).group(1)
                #print(href)
                books.append(bookID)
    return list(set(books))

def book_page(url):
    global authors
    #url just like https://book.douban.com/subject/27079142/
    header = {'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
    req = urllib.request.Request(url,headers=header)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html,'lxml')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if re.match(r'https://book.douban.com/author/[0-9]*/',href):
            authors.append(href)
            print(href)
            
#返回一位作家评价最高的前五本书[书名，评分，评价人数]
def get_booklist(author_id):
    booklist = []
    url = 'https://book.douban.com/author/'+str(author_id)+'/books?sortby=collect&format=pic'
    header = {'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
    req = urllib.request.Request(url,headers=header)   
    with urllib.request.urlopen(req,timeout=30) as html:
        html = html.read()
        soup = BeautifulSoup(html,'lxml')
        items = soup.find(class_='grid_view').find('ul').find_all('dd')
        for counter,item in enumerate(items):
            book_name = item.find('h6').a.text
            star = item.find(class_='star clearfix').find_all('span')[1].text
            evStr = item.find(class_='star clearfix').find_all('span')[2].text
            evNum = re.search(u'([0-9]*)人评价'.encode('utf-8'),evStr.encode('utf-8')).group(1).decode('utf-8')
            
            if star!='' :
                book = (book_name,star,evNum)
                #print(book_name,star,evNum)
                booklist.append(book)
            if counter==5 :
                break
                
        return booklist

def get_author():
    global authors
    authors = set(authors)
    header = {'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
    for url in authors:
        print('url :',url)
        req = urllib.request.Request(url,headers=header)

        with urllib.request.urlopen(req,timeout=30) as html:
            html = html.read()
            soup = BeautifulSoup(html,'lxml')
            name = soup.find(id='content').h1.text
            info = soup.find(class_='info').ul.find_all('li')
            gender = re.search(u'男|女'.encode('utf-8'),info[0].text.encode('utf-8')).group().decode('utf-8')
            birthday = re.search(r'[0-9].*',info[1].text).group()
            country = re.search(u'(.*):(.*)'.encode('utf-8'),info[2].text.encode('utf-8'),re.S).group(2).decode('utf-8')
            country = country.strip()
    
            author_id = re.search(r'(.*)author/(.*)/',url).group(2)
            
            
            out = open('author.csv', 'w+', newline='')
            csv_writer = csv.writer(out, dialect='excel')
            
            print(author_id,name,gender,birthday,country,get_booklist(author_id))
            csv_writer.writerow([author_id,name,gender,birthday,country,get_booklist(author_id)])
            #out.close()

def search_book():
    global books
    for counter,book in enumerate(books):
        url = 'https://book.douban.com/subject/'+book+'/'
        header = {'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
        req = urllib.request.Request(url,headers=header)
        with urllib.request.urlopen(req,timeout=30) as html:
            html = html.read()
            soup = BeautifulSoup(html,'lxml')
            recBooks = soup.find(id='db-rec-section').find_all('dd')
            for item in recBooks:
                bookLink = item.a.get('href')
                bookID = re.search(r'https://book.douban.com/subject/([0-9]*)/$',bookLink).group(1)
                if bookID not in books:
                    books.append(bookID)
                    out = open('book.csv', 'w+', newline='')
                    csv_writer = csv.writer(out, dialect='excel')
                    csv_writer.writerow(bookID)
        if counter == 100:
            break
    
if __name__=='__main__':
    search_book()
    print(books)

URLError: <urlopen error _ssl.c:761: The handshake operation timed out>

In [ ]:
list = [1,2,2,3]


In [57]:
authors

NameError: name 'authors' is not defined

In [56]:
import requests
import urllib
import re
import threading
from bs4 import *
import socket
import time
import csv
header = {'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
url = 'https://book.douban.com/author/1071735/'
req = urllib.request.Request(url,headers=header)

with urllib.request.urlopen(req,timeout=30) as html:
    html = html.read()
    soup = BeautifulSoup(html,'lxml')
    name = soup.find(id='content').h1.text
    info = soup.find(class_='info').ul.find_all('li')
    gender = re.search(u'男|女'.encode('utf-8'),info[0].text.encode('utf-8')).group().decode('utf-8')
    birthday = re.search(r'[0-9].*',info[1].text).group()
    country = re.search(u'(.*):(.*)'.encode('utf-8'),info[2].text.encode('utf-8'),re.S).group(2).decode('utf-8')
    country = country.strip()
    
    author_id = re.search(r'(.*)author/(.*)/',url).group(2)
    #introduction = soup.find(class_='hd').find(class_='bd').text
    print(author_id,name,gender,birthday,country)
    out = open('author.csv', 'w', newline='')
    csv_writer = csv.writer(out, dialect='excel')
    csv_writer.writerow([author_id,name,gender,birthday,country])
    #out.close()


1071735 阿拉文德·阿迪加 Aravind Adiga 男 1974 印度


In [21]:
re.search(u'(.*)是(.*)',u'我是你爸爸').group(2)

'你爸爸'

In [ ]:
httpResult = []
httpsResult = []
for page in range(1,5):
    IPurl = 'http://www.xicidaili.com/nn/%s' %page
    rIP=requests.get(IPurl,headers=headers)
    IPContent=rIP.text
    soupIP = BeautifulSoup(IPContent,"html5lib")
    trs = soupIP.find_all('tr')
    for tr in trs[1:]:
        tds = tr.find_all('td')
        ip = tds[2].text.strip()
        port = tds[3].text.strip()
        protocol = tds[6].text.strip()
        if protocol == 'HTTP':
            st = 'http://' + ip + ':' + port
            httpResult.append(st)
        elif protocol =='HTTPS':
            st = 'https://' + ip + ':' + port
            httpsResult.append(st)

proxies = set(httpResult)+set(httpsResult)
headers = {
"user-agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36",
}

r = requests.get('https://book.douban.com/author/310611/',headers=headers,proxies=proxies)
content = r.text

In [54]:
import requests
import urllib
import re
import threading
from bs4 import *
import socket
import time
import csv

#返回一位作家评价最高的前五本书[书名，评分，评价人数]
def get_booklist(author_id):
    booklist = []
    url = 'https://book.douban.com/author/'+str(author_id)+'/books?sortby=collect&format=pic'
    header = {'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
    req = urllib.request.Request(url,headers=header)   
    with urllib.request.urlopen(req,timeout=30) as html:
        html = html.read()
        soup = BeautifulSoup(html,'lxml')
        items = soup.find(class_='grid_view').find('ul').find_all('dd')
        for counter,item in enumerate(items):
            book_name = item.find('h6').a.text
            star = item.find(class_='star clearfix').find_all('span')[1].text
            evStr = item.find(class_='star clearfix').find_all('span')[2].text
            evNum = re.search(u'([0-9]*)人评价'.encode('utf-8'),evStr.encode('utf-8')).group(1).decode('utf-8')
            
            if star!='' :
                book = (book_name,star,evNum)
                print(book_name,star,evNum)
                booklist.append(book)
            if counter==5 :
                break
                
        return booklist
            
if __name__=='__main__':
    get_booklist()
        

避暑 7.8 221
文学“爆炸”亲历记 7.6 71
污秽的夜鸟 8.9 31
周末逸事 6.4 11


In [39]:
arr = ['a','b','c']

for i,a in enumerate(arr):
    print(i,a)

0 a
1 b
2 c


In [49]:
out = open('author.csv', 'w', newline='')
csv_writer = csv.writer(out, dialect='excel')
    
csv_writer.writerow([1,2,3,4,[['a','b','c'],['a','b','c'],['a','b','c'],['a','b','c']]])

80

In [ ]:
def search_book(books):
    global books
    for counter,book in enumerate(books):
        url = 'https://book.douban.com/subject/'+books+'/'
        header = {'User-Agent':'User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'}
        req = urllib.request.Request(url,headers=header)
        with urllib.request.urlopen(req,timeout=30) as html:
            html = html.read()
            soup = BeautifulSoup(html,'lxml')
            recBooks = soup.find(id='db-rec-section').find_all('dd')
            for item in recBooks:
                bookLink = item.a.get('href')
                bookID = re.search(r'https://book.douban.com/subject/([0-9]*)/$',bookLink).group(1)
                if bookID not in books:
                    books.append(bookID)
                    out = open('book.csv', 'w+', newline='')
                    csv_writer = csv.writer(out, dialect='excel')
                    csv_writer.writerow(bookID)
        if counter == 100:
            break
    

In [68]:

list = [1,2,3]


for i,item in enumerate(list):
    list.append(item)
    if i==100:
        break

In [69]:
list

[1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2,
 3,
 1,
 2]